# 📊 Code Interpreter with Boto3: Data Processing & File Operations

Welcome to the Code Interpreter module using AWS Bedrock AgentCore with boto3! This notebook demonstrates both data processing/analysis and file operations using direct boto3 API calls.

## 🎯 Learning Objectives

In this notebook, you'll learn how to:

- 🔧 Set up and use AgentCore Code Interpreter with boto3
- 📈 Perform statistical calculations and data analysis
- 📁 Execute file operations (read, write, process)
- 🔄 Process data using JavaScript secure sandboxes

## 📋 Prerequisites

- AWS account with access to Amazon Bedrock AgentCore
- Proper IAM permissions for AgentCore services
- Python 3.10+ environment

Let's begin! 🚀

## Amazon Bedrock AgentCore Code Interpreter

The Amazon Bedrock AgentCore Code Interpreter enables AI agents to write and execute code securely in sandbox environments, enhancing their accuracy and expanding their ability to solve complex end-to-end tasks. This is critical in Agentic AI applications where the agents may execute arbitrary code that can lead to data compromise or security risks.

Key features:
- Secure code execution in isolated sandboxes
- Support for multiple programming languages (Python, JavaScript)
- Large file support (up to 100 MB inline, 5 GB via S3)
- CloudTrail logging capabilities
- Network access control

![Amazon Bedrock AgentCore Code Interpreter Architecture](../../../media/agent_core_code_interpreter_arch.png)

## 🔧 Section 1: Environment Setup

In [ ]:
# 🔧 Environment Setup

import sys
import subprocess

# Check Python version
python_version = sys.version_info
print(f"Python version: {python_version.major}.{python_version.minor}.{python_version.micro}")

if python_version.major == 3 and python_version.minor < 10:
    print("⚠️ Warning: Python 3.10+ is recommended. Some features may not work.")

# Install required packages if not already installed
required_packages = ['boto3']

for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

print("\n✅ Environment setup complete!")

In [ ]:
# Import dependencies
import boto3
import json
import time
from typing import Dict, Any, List, Optional

print("✅ Dependencies imported successfully!")

In [ ]:
# Initialize AWS clients using default session
session = boto3.Session()
region_name = session.region_name

# Verify credentials and region are working
try:
    credentials = session.get_credentials()
    print(f"✅ Credentials loaded successfully")
    print(f"✅ Region: {region_name}")
    
    # Get the current role ARN if available
    sts = session.client('sts')
    identity = sts.get_caller_identity()
    print(f"📋 Current identity: {identity['Arn']}")
    
    if 'SageMaker' in identity['Arn']:
        print("\n⚠️  Running in SageMaker - Please ensure your execution role has the required permissions.")
        print("   See the 'IAM Permissions Setup' section below if you encounter permission errors.")
        
except Exception as e:
    print(f"❌ Error with AWS credentials: {e}")
    raise

# Initialize AgentCore client
bedrock_agentcore_client = session.client(
    'bedrock-agentcore',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore.{region_name}.amazonaws.com"
)

print("✅ AWS clients initialized successfully!")

## ⚠️ IAM Permissions Setup (For SageMaker Users)

If you're running this notebook in **SageMaker** and encounter permission errors, you need to add the following permissions to your SageMaker execution role:

### Required Permissions

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock-agentcore:StartCodeInterpreterSession",
        "bedrock-agentcore:StopCodeInterpreterSession",
        "bedrock-agentcore:InvokeCodeInterpreter"
      ],
      "Resource": "arn:aws:bedrock-agentcore:*:aws:code-interpreter/*"
    }
  ]
}
```

### How to Add Permissions

1. **Go to IAM Console**: https://console.aws.amazon.com/iam/
2. **Find your SageMaker role**: Search for `AmazonSageMakerServiceCatalogProductsUseRole` or similar
3. **Add inline policy**: Click "Add permissions" → "Create inline policy"
4. **Use JSON editor**: Paste the policy above
5. **Name the policy**: e.g., "BedrockAgentCoreCodeInterpreterAccess"
6. **Save**: Click "Create policy"

After adding these permissions, restart your kernel and try again.

---

## 🛠️ Helper Functions

In [ ]:
def execute_code(code: str, language: str = "python", session_name: str = "code-session") -> Dict[str, Any]:
    """
    Execute code using boto3 direct API calls to AgentCore Code Interpreter.
    
    Args:
        code: The code to execute
        language: Programming language (javascript or python)
        session_name: Name for the code interpreter session
    
    Returns:
        Dictionary with execution results
    """
    try:
        # Start a code interpreter session
        session_response = bedrock_agentcore_client.start_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            name=session_name,
            sessionTimeoutSeconds=300
        )
        session_id = session_response["sessionId"]
        print(f"✅ Started session: {session_id}")
        
        # Execute the code
        execute_response = bedrock_agentcore_client.invoke_code_interpreter(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id,
            name="executeCode",
            arguments={
                "language": language,
                "code": code
            }
        )
        
        # Process the response stream
        results = []
        for event in execute_response['stream']:
            if 'result' in event:
                result = event['result']
                results.append(result)
                
                # Print text output
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(f"Output: {content_item['text']}")
        
        # Clean up session
        bedrock_agentcore_client.stop_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id
        )
        print(f"✅ Session {session_id} stopped")
        
        return {
            "success": True,
            "session_id": session_id,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

In [ ]:
def call_tool(session_id: str, tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to call Code Interpreter tools using boto3"""
    response = bedrock_agentcore_client.invoke_code_interpreter(
        codeInterpreterIdentifier="aws.codeinterpreter.v1",
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    
    for event in response["stream"]:
        if "result" in event:
            return event["result"]
    return {}

## 📊 Example 1: Data Processing & Analysis

Let's demonstrate statistical analysis using both JavaScript and Python.

### JavaScript Statistical Analysis

In [ ]:
# JavaScript code for temperature data analysis
javascript_stats_code = """
// Temperature data analysis
console.log('Starting temperature data analysis...');

// Daily temperatures in Celsius
const temperatures = [22.5, 24.1, 23.8, 25.2, 21.9, 26.3, 23.7];
const days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'];

// Calculate statistics
function calculateStats(data) {
    const sum = data.reduce((a, b) => a + b, 0);
    const mean = sum / data.length;
    const min = Math.min(...data);
    const max = Math.max(...data);
    
    // Calculate variance and standard deviation
    const variance = data.reduce((acc, val) => {
        return acc + Math.pow(val - mean, 2);
    }, 0) / data.length;
    const stdDev = Math.sqrt(variance);
    
    return {
        count: data.length,
        sum: sum.toFixed(2),
        mean: mean.toFixed(2),
        min: min.toFixed(2),
        max: max.toFixed(2),
        variance: variance.toFixed(2),
        stdDev: stdDev.toFixed(2),
        range: (max - min).toFixed(2)
    };
}

const stats = calculateStats(temperatures);

// Display results
console.log('=== TEMPERATURE STATISTICS ===');
console.log('Number of readings: ' + stats.count);
console.log('Average temperature: ' + stats.mean + '°C');
console.log('Minimum temperature: ' + stats.min + '°C');
console.log('Maximum temperature: ' + stats.max + '°C');
console.log('Temperature range: ' + stats.range + '°C');
console.log('Standard deviation: ' + stats.stdDev + '°C');

// Display daily readings
console.log('=== DAILY READINGS ===');
days.forEach((day, index) => {
    const temp = temperatures[index];
    const diff = (temp - stats.mean).toFixed(2);
    const sign = diff >= 0 ? '+' : '';
    console.log(`${day}: ${temp}°C (${sign}${diff}°C from average)`);
});

console.log('Analysis complete!');
stats;
"""

print("JavaScript code prepared for execution")

In [ ]:
# Execute JavaScript statistical analysis
print("🚀 Executing JavaScript temperature analysis...\n")
js_result = execute_code(
    javascript_stats_code, 
    language="javascript",
    session_name="js-stats-session"
)
print(f"\n✅ Execution completed: {js_result['success']}")

## 📁 Example 2: File Operations

Now let's demonstrate file operations including creating, reading, and processing files.

In [ ]:
# Prepare sample data for file operations
sample_csv_data = """name,age,city,occupation,salary
Alice Johnson,28,New York,Software Engineer,95000
Bob Smith,35,San Francisco,Data Scientist,120000
Carol Williams,42,Chicago,Product Manager,110000
David Brown,31,Austin,DevOps Engineer,105000
Emma Davis,26,Seattle,UX Designer,85000
Frank Miller,39,Boston,Backend Developer,100000
Grace Lee,33,Denver,Frontend Developer,95000
Henry Wilson,29,Portland,QA Engineer,75000"""

# Python script to analyze the CSV data
analysis_script = """import csv
import json
from collections import defaultdict

# Read and analyze the CSV data
with open('employees.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    data = list(csv_reader)

# Convert numeric fields
for row in data:
    row['age'] = int(row['age'])
    row['salary'] = int(row['salary'])

# Calculate statistics
total_employees = len(data)
ages = [row['age'] for row in data]
salaries = [row['salary'] for row in data]

avg_age = sum(ages) / len(ages)
avg_salary = sum(salaries) / len(salaries)

# Group by city
by_city = defaultdict(list)
for row in data:
    by_city[row['city']].append(row)

# Group by occupation
by_occupation = defaultdict(list)
for row in data:
    by_occupation[row['occupation']].append(row)

# Create analysis report
report = {
    'summary': {
        'total_employees': total_employees,
        'average_age': round(avg_age, 1),
        'average_salary': round(avg_salary, 2),
        'age_range': {'min': min(ages), 'max': max(ages)},
        'salary_range': {'min': min(salaries), 'max': max(salaries)}
    },
    'by_city': {city: len(employees) for city, employees in by_city.items()},
    'by_occupation': {occ: len(employees) for occ, employees in by_occupation.items()},
    'top_earners': sorted(data, key=lambda x: x['salary'], reverse=True)[:3]
}

# Save report to JSON
with open('analysis_report.json', 'w') as f:
    json.dump(report, f, indent=2)

# Display results
print('=== EMPLOYEE DATA ANALYSIS ===')
print('Total employees: ' + str(report['summary']['total_employees']))
print('Average age: ' + str(report['summary']['average_age']) + ' years')
print('Average salary: $' + '{:,.2f}'.format(report['summary']['average_salary']))
print('\\nEmployees by city:')
for city, count in report['by_city'].items():
    print('  ' + city + ': ' + str(count))
print('\\nTop 3 earners:')
for i, emp in enumerate(report['top_earners'], 1):
    print('  ' + str(i) + '. ' + emp['name'] + ' - $' + '{:,}'.format(emp['salary']))
print('\\nReport saved to analysis_report.json')
"""

# Configuration file
config_json = json.dumps({
    "analysis": {
        "version": "1.0.0",
        "date": "2024-01-14",
        "author": "Data Analysis Team"
    },
    "settings": {
        "output_format": "json",
        "include_charts": False,
        "decimal_places": 2
    }
}, indent=2)

print("✅ Sample data prepared for file operations")

In [ ]:
# Start a new session for file operations
print("🚀 Starting Code Interpreter session for file operations...")

try:
    session_response = bedrock_agentcore_client.start_code_interpreter_session(
        codeInterpreterIdentifier="aws.codeinterpreter.v1",
        name="file-ops-session",
        sessionTimeoutSeconds=300
    )
    file_session_id = session_response["sessionId"]
    print(f"✅ Session started: {file_session_id}")
except Exception as e:
    print(f"❌ Error starting session: {e}")

In [ ]:
# Create files in the session
files_to_create = [
    {
        "path": "employees.csv",
        "text": sample_csv_data
    },
    {
        "path": "analyze.py",
        "text": analysis_script
    },
    {
        "path": "config.json",
        "text": config_json
    }
]

print("📝 Writing files to Code Interpreter session...")
try:
    write_result = call_tool(file_session_id, "writeFiles", {"content": files_to_create})
    print(f"✅ Files written successfully")
    if 'content' in write_result:
        for item in write_result['content']:
            if item['type'] == 'text':
                print(f"  {item['text']}")
except Exception as e:
    print(f"❌ Error writing files: {e}")

In [ ]:
# List files to verify they were created
print("\n📋 Listing files in session...")
try:
    list_result = call_tool(file_session_id, "listFiles", {"path": ""})
    print("Files in session:")
    if 'content' in list_result:
        for item in list_result['content']:
            if item['description'] == 'File':
                print(f"  {item['name']}")
except Exception as e:
    print(f"❌ Error listing files: {e}")

In [ ]:
# Execute the analysis script
print("\n🔧 Executing analysis script...")
try:
    exec_result = call_tool(file_session_id, "executeCode", {
        "code": analysis_script,
        "language": "python",
        "clearContext": True
    })
    print("📊 Analysis results:")
    if 'content' in exec_result:
        for item in exec_result['content']:
            if item['type'] == 'text':
                print(item['text'])
except Exception as e:
    print(f"❌ Error executing script: {e}")

In [ ]:
# Read the generated report file
print("\n📖 Reading generated analysis report...")
try:
    read_result = call_tool(file_session_id, "readFiles", {"paths": ["analysis_report.json"]})
    print("Analysis Report Content:")
    if 'content' in read_result:
        for item in read_result['content']:
            print(item)
            if item['resource'] == 'text':
                # Parse and pretty print the JSON
                try:
                    report_data = json.loads(item['text'])
                    print(json.dumps(report_data, indent=2))
                except:
                    print(item['text'])
except Exception as e:
    print(f"❌ Error reading report: {e}")

In [ ]:
# Clean up the file operations session
print("\n🧹 Cleaning up file operations session...")
try:
    bedrock_agentcore_client.stop_code_interpreter_session(
        codeInterpreterIdentifier="aws.codeinterpreter.v1",
        sessionId=file_session_id
    )
    print(f"✅ Session {file_session_id} stopped successfully")
except Exception as e:
    print(f"❌ Error stopping session: {e}")

## 📝 Summary

In this notebook, we've successfully demonstrated:

### 1. **Data Processing & Analysis**
   - **JavaScript**: Temperature data analysis with statistical calculations
   - **Python**: Student score analysis with comprehensive statistics and grade distribution
   - Both examples showed mean, median, variance, standard deviation calculations

### 2. **File Operations**
   - Creating multiple files (CSV, Python script, JSON config)
   - Listing files in the Code Interpreter session
   - Executing scripts that read and process files
   - Reading generated output files

### 🔗 Additional Resources

- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/)
- [Code Interpreter API Reference](https://docs.aws.amazon.com/bedrock-agentcore/latest/APIReference/)
- [SageMaker Setup Guide](../SAGEMAKER_SETUP.md)